In [12]:
import os
import json
import collections
import matplotlib.pyplot as plt
import numpy as np
import collections

In [13]:
cdir = %pwd
cdir

'/Users/kkagawa/localdoc/github/earthquake'

In [14]:
data_path = os.path.join(cdir, 'data/data_test.json')
with open(data_path) as f:
    data = json.load(f)
word_list_tmp = []
for i in range(len(data)):
    word_list_tmp.append(data[i]['word'].lower())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# SeleniumでJavaScriptを動かす

In [ ]:
from time import sleep                 # スリープを使うために必要
from selenium import webdriver         # Webブラウザを自動操作する（python -m pip install selenium)
import chromedriver_binary             # パスを通すためのコード
from selenium.webdriver.common.by import By

In [ ]:
# テストコード
driver = webdriver.Chrome()            # Chromeを準備
driver.get('https://www.google.com/')  # Googleを開く
time.sleep(5)                          # 5秒間待機
driver.quit()                          # ブラウザを閉じる

In [ ]:
# Googleニュースサイトを確認
browser = webdriver.Chrome() #ブラウザの立ち上がる
browser.get('https://news.google.com/topstories?tab=wn&hl=ja&gl=JP&ceid=JP:ja') #google newsを開く
top = browser.find_element_by_class_name('DY5T1d') #クラスでトップ記事の見出しを取得

top.text #タグの中身の文字を取得

<ipython-input-6-5ed283419a6f>:3: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  top = browser.find_element_by_class_name('DY5T1d') #クラスでトップ記事の見出しを取得


'東部の攻防 ゼレンスキー大統領 “部隊は踏みとどまっている”'

In [ ]:
# 気象庁のページを確認
# クリックされずに非表示になっていることを確認
url = 'https://www.data.jma.go.jp/svd/eqev/data/daily_map/20220430.html'
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

In [ ]:
# クリックして表示されたものを保存
date = '20220618'
# date = '20230510'
url = f'https://www.data.jma.go.jp/svd/eqev/data/daily_map/{date}.html'
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
# driver.quit()

submenu = driver.find_element(by=By.CLASS_NAME, value='subMenu')
driver.execute_script("arguments[0].style.display='block';", submenu)
data_org = submenu.text

In [ ]:
# data
print(data_org)

年   月 日 時 分 秒    緯度       経度       深さ(km)  Ｍ   震央地名
-----------------------------------------------------------------------------------------
2022  6 18 00:00 21.6  45°20.0'N 141°39.0'E   34     0.2  宗谷地方北部                  
2022  6 18 00:02 19.6  24° 6.7'N 123°43.5'E   21     3.1  西表島付近                   
2022  6 18 00:04  4.2  37°50.8'N 141°47.3'E   41     1.0  福島県沖                    
2022  6 18 00:04 13.3  39° 5.0'N 142°10.6'E   41     0.8  岩手県沖                    
2022  6 18 00:06  5.2  32° 1.6'N 131°56.4'E   27     1.1  日向灘                     
2022  6 18 00:06  6.3  36°17.7'N 138° 4.3'E    4    -0.6  長野県中部                   
2022  6 18 00:07 54.1  39°42.3'N 141°52.3'E   46     0.0  岩手県沿岸北部                 
2022  6 18 00:08 55.4  34°50.7'N 137°52.7'E   18     1.4  静岡県西部                   
2022  6 18 00:09 10.1  38°48.4'N 141°44.7'E   85     0.7  宮城県沖                    
2022  6 18 00:14 26.4  35°54.1'N 138° 8.2'E   10    -0.6  長野県南部                   

2022  6 18 00:14 32.5  27°5

# 複数日時のデータ取得

In [35]:
from time import sleep                 # スリープを使うために必要
from selenium import webdriver         # Webブラウザを自動操作する（python -m pip install selenium)
import chromedriver_binary             # パスを通すためのコード
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
import re

In [36]:
# 関数化
def get_data_earthquake(date):
    # クリックして表示されたものを保存
    url = f'https://www.data.jma.go.jp/svd/eqev/data/daily_map/{date}.html'
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    # driver.quit()

    submenu = driver.find_element(by=By.CLASS_NAME, value='subMenu')
    driver.execute_script("arguments[0].style.display='block';", submenu)
    data_org = submenu.text

    data = data_org
    data = re.sub(r'---+', '', data_org)
    data = data.replace('\n\n', '\n')
    data = data.replace(r':', '\t')
    data = data.replace(r'° ','°')
    data = data.replace('°', '\t')
    data = data.replace("'N", '')
    data = data.replace("'E", '')
    data = data.replace('緯度', '緯度(度)\t緯度(分)')
    data = data.replace('経度', '経度(度)\t経度(分)')

    df_org = pd.DataFrame(np.array(data.split()).reshape(-1,13))
    df_column = df_org.iloc[0].to_list()
    df = df_org.drop(df_org.index[[0]])
    df.columns = df_column

    # 保存
    df.to_csv(f'./data/{date}.csv', index=False)

In [42]:
from datetime import datetime
from datetime import timedelta

# 日付条件の設定
strdt = datetime.strptime("20230401", '%Y%m%d')  # 開始日
enddt = datetime.strptime("20230531", '%Y%m%d')  # 終了日

# 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります

datelist = []
for i in range(days_num):
    datelist.append((strdt + timedelta(days=i)).strftime("%Y%m%d"))

# 確認
for d in datelist:
    print(d)


20230401
20230402
20230403
20230404
20230405
20230406
20230407
20230408
20230409
20230410
20230411
20230412
20230413
20230414
20230415
20230416
20230417
20230418
20230419
20230420
20230421
20230422
20230423
20230424
20230425
20230426
20230427
20230428
20230429
20230430
20230501
20230502
20230503
20230504
20230505
20230506
20230507
20230508
20230509
20230510
20230511
20230512
20230513
20230514
20230515
20230516
20230517
20230518
20230519
20230520
20230521
20230522
20230523
20230524
20230525
20230526
20230527
20230528
20230529
20230530
20230531


In [43]:
for date in datelist:
    get_data_earthquake(date)
    sleep(10)

# データの整形

In [ ]:
import pandas as pd
import re


In [ ]:
data = data_org
data = re.sub(r'---+', '', data_org)
data = data.replace('\n\n', '\n')
data = data.replace(r':', '\t')
data = data.replace(r'° ','°')
data = data.replace('°', '\t')
data = data.replace("'N", '')
data = data.replace("'E", '')
data = data.replace('緯度', '緯度(度)\t緯度(分)')
data = data.replace('経度', '経度(度)\t経度(分)')
print(data)

NameError: name 'data_org' is not defined

In [ ]:
df_org = pd.DataFrame(np.array(data.split()).reshape(-1,13))
df_column = df_org.iloc[0].to_list()
df = df_org.drop(df_org.index[[0]])
df.columns = df_column
df

,年,月,日,時,分,秒,緯度(度),緯度(分),経度(度),経度(分),深さ(km),Ｍ,震央地名
1,2022,4,30,00,00,31.7,27,24.6,128,50.7,31,1.6,沖縄本島近海
2,2022,4,30,00,03,53.0,44,39.7,144,21.7,267,2.5,網走沖
3,2022,4,30,00,04,1.1,34,44.7,133,0.2,18,0.5,広島県北部
4,2022,4,30,00,05,15.7,39,3.4,140,51.9,10,0.6,岩手県内陸南部
5,2022,4,30,00,05,35.4,36,9.0,141,54.2,12,1.5,茨城県沖
...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,2022,4,30,23,47,53.8,33,32.6,135,25.3,29,0.7,和歌山県南方沖
699,2022,4,30,23,47,55.5,33,31.4,133,54.2,30,0.2,高知県東部
700,2022,4,30,23,48,7.7,35,21.2,133,51.4,5,-0.4,鳥取県中部
701,2022,4,30,23,49,4.4,33,34.3,135,25.8,27,0.3,和歌山県南方沖


In [ ]:
# 保存
df.to_csv('./data/220430.csv', index=False)

In [ ]:
df1 = pd.read_csv('./data/220430.csv')
df1

,年,月,日,時,分,秒,緯度(度),緯度(分),経度(度),経度(分),深さ(km),Ｍ,震央地名
0,2022,4,30,0,0,31.7,27,24.6,128,50.7,31,1.6,沖縄本島近海
1,2022,4,30,0,3,53.0,44,39.7,144,21.7,267,2.5,網走沖
2,2022,4,30,0,4,1.1,34,44.7,133,0.2,18,0.5,広島県北部
3,2022,4,30,0,5,15.7,39,3.4,140,51.9,10,0.6,岩手県内陸南部
4,2022,4,30,0,5,35.4,36,9.0,141,54.2,12,1.5,茨城県沖
...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,2022,4,30,23,47,53.8,33,32.6,135,25.3,29,0.7,和歌山県南方沖
698,2022,4,30,23,47,55.5,33,31.4,133,54.2,30,0.2,高知県東部
699,2022,4,30,23,48,7.7,35,21.2,133,51.4,5,-0.4,鳥取県中部
700,2022,4,30,23,49,4.4,33,34.3,135,25.8,27,0.3,和歌山県南方沖


## 日本地図

In [ ]:
import pandas as pd
import numpy as np
import folium # pip install folium
import plotly.express as px

In [ ]:
df = pd.read_csv('data/20231231.csv')
df.head()
df.loc[3,"緯度(度)"]
df_mod = df.copy()
# 緯度(度)と緯度(分)を組み合わせて新しい'lat'列を作成
df_mod['lat'] = df['緯度(度)'] + df['緯度(分)'] / 60
df_mod['lon'] = df['経度(度)'] + df['経度(分)'] / 60
df_mod.head()

,年,月,日,時,分,秒,緯度(度),緯度(分),経度(度),経度(分),深さ(km),Ｍ,震央地名,lat,lon
0,2023,12,31,0,0,21.3,36,1.4,137,31.4,6,1.0,岐阜県飛騨地方,36.023333,137.523333
1,2023,12,31,0,6,34.4,36,49.9,138,45.9,5,0.7,新潟県中越地方,36.831667,138.765000
2,2023,12,31,0,8,30.9,36,49.1,140,29.1,9,0.1,福島県中通り,36.818333,140.485000
3,2023,12,31,0,9,53.5,37,32.8,137,14.7,14,1.3,能登半島沖,37.546667,137.245000
4,2023,12,31,0,12,31.6,41,18.3,141,3.7,7,-0.3,青森県下北地方,41.305000,141.061667


In [ ]:
fig = px.scatter_mapbox(
    # データフレームおよび緯度・経度の設定
    data_frame=df_mod,
    lat="lat",
    lon="lon",
    #カーソルを当てるとホバーする情報を与えます
    hover_data=["震央地名","Ｍ"],
    zoom=4,
    height=700,
    width=700)
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [34]:
# 特定の地域のデータを抽出
# df = pd.read_csv('data/20220430.csv')
df = pd.read_csv('data/20220610.csv')
# df = pd.read_csv('data/20231231.csv')

df_mod = df.copy()
# 緯度(度)と緯度(分)を組み合わせて新しい'lat'列を作成
df_mod['緯度'] = df['緯度(度)'] + df['緯度(分)'] / 60
df_mod['経度'] = df['経度(度)'] + df['経度(分)'] / 60

selected_regions = ["石川県能登地方", "能登半島沖"]
filtered_df = df_mod[df_mod['震央地名'].isin(selected_regions)]
print(len(filtered_df))
# display(filtered_df.head())

fig = px.scatter_mapbox(
    # データフレームおよび緯度・経度の設定
    data_frame=filtered_df,
    lat='緯度',
    lon='経度',
    #カーソルを当てるとホバーする情報を与えます
    hover_data=['深さ(km)','Ｍ','震央地名'],
    zoom=10,
    height=700,
    width=700)
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# 中心座標の指定
fig.update_layout(mapbox_center={'lat': 37.3, 'lon': 137.5})
fig.show()

80


In [ ]:
#サイズを指定する
folium_figure = folium.Figure(width=600, height=600)
# folium_map = folium.Map([35, 139],zoom_start=4.5).add_to(folium_figure)
folium_map = folium.Map([37,137],zoom_start=8).add_to(folium_figure)
i=3
folium.Marker(location=[df.loc[i,"緯度(度)"],df.loc[i,"経度(度)"]]).add_to(folium_map)
folium_figure

# データ分析

In [11]:
import pandas as pd

In [34]:
# CSVファイルを読み込む
# df = pd.read_csv('data/20220430.csv')
# df = pd.read_csv('data/20220601.csv')
# df = pd.read_csv('data/20220619.csv')
df = pd.read_csv('data/20231201.csv')
# df = pd.read_csv('data/20231231.csv')
df.head()

,年,月,日,時,分,秒,緯度(度),緯度(分),経度(度),経度(分),深さ(km),Ｍ,震央地名
0,2023,12,1,0,0,14.5,36,16.3,140,50.5,45,0.9,茨城県沖
1,2023,12,1,0,7,3.5,34,20.6,140,18.0,75,1.9,房総半島南方沖
2,2023,12,1,0,11,27.3,32,22.7,130,41.2,9,1.0,熊本県熊本地方
3,2023,12,1,0,14,27.5,42,21.6,144,46.6,22,1.7,釧路沖
4,2023,12,1,0,26,28.5,37,24.8,141,42.7,41,1.4,福島県沖


In [35]:
# 特定の地域のデータを抽出
selected_regions = ["石川県能登地方", "能登半島沖"]
filtered_df = df[df['震央地名'].isin(selected_regions)]
print(len(filtered_df))
display(filtered_df)

11


,年,月,日,時,分,秒,緯度(度),緯度(分),経度(度),経度(分),深さ(km),Ｍ,震央地名
76,2023,12,1,3,29,5.1,37,31.1,137,19.1,13,2.4,石川県能登地方
88,2023,12,1,4,7,8.6,37,31.9,137,19.4,14,1.3,石川県能登地方
100,2023,12,1,4,40,6.5,37,31.8,137,19.8,14,1.2,石川県能登地方
114,2023,12,1,5,37,41.8,37,32.8,137,15.6,13,-,能登半島沖
115,2023,12,1,5,37,42.4,37,32.5,137,14.9,13,1.4,能登半島沖
164,2023,12,1,8,42,52.7,37,29.9,137,15.6,13,2.1,石川県能登地方
286,2023,12,1,14,41,34.9,37,33.6,137,20.4,12,1.6,能登半島沖
317,2023,12,1,16,32,51.2,37,30.7,137,12.7,13,1.2,石川県能登地方
331,2023,12,1,17,43,12.1,37,30.1,137,12.9,13,1.6,石川県能登地方
405,2023,12,1,21,13,4.3,37,31.7,137,11.8,13,0.9,能登半島沖
